<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [3]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [5]:
data_train.head(3)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, author...","Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murde..."
2,00027e965c8264c35cc1bc55556db388da82b07f,"A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he r...","Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titl..."


In [6]:
data = pd.concat([data_train, data_test, data_validation ])
train_df = data

In [7]:
train_df.shape

(311971, 3)

In [8]:
sample=data_train.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
['In the eastern Ukrainian city of Luhansk, makeshift barricades, concertina wire and masked men in camouflage greeted visitors to the regional administration building Wednesday. Seized by armed men Tuesday, the building in Ukraine\'s restive Donetsk region is just the latest to fall under the control of pro-Russian militants. Government sites across more than a dozen towns and cities in Donetsk remain occupied, despite an international deal agreed to earlier this month that called for illegal armed groups to disarm and go home. And the militias, resolutely defiant, show no signs of changing their stance. In the foyer of the Luhansk government building -- outside which pro-Russian flags now fly -- more armed men, sandbags and wire surrounded a desk through which access to the rest of the building was controlled. A handful of employees waited, looking very uncomfortable. At a briefing inside for reporters, a man who described himself as the press secretary for the headqu

Checking for Null values


In [9]:
data_train.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [10]:
data_train.duplicated(subset=['article','highlights']).sum()

3098

In [11]:
data_train=data_train.drop_duplicates(subset=['article','highlights'])
data_train.shape

(284015, 3)

In [12]:
data_train.duplicated(subset=['article','highlights']).sum()

0

In [14]:
data_train['article'][:10]

0    By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundr...
1    (CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, author...
2    A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he r...
3    (CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prio...
4    Fleetwood are the only team still to have a 100% record in Sky Bet League One as a 2-0 win over Scunthorpe sent Graham Alexander’s men top of the table. The Cod Army are playi

In [16]:
data_train['article'] = data_train['article'].str.lower()
data_train['highlights'] = data_train['highlights'].str.lower()

data_train.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"by . associated press . published: . 14:11 est, 25 october 2013 . | . updated: . 15:36 est, 25 october 2013 . the bishop of the fargo catholic diocese in north dakota has exposed potentially hundr...","bishop john folda, of north dakota, is taking time off after being diagnosed .\nhe contracted the infection through contaminated food in italy .\nchurch members in fargo, grand forks and jamestown..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(cnn) -- ralph mata was an internal affairs lieutenant for the miami-dade police department, working in the division that investigates allegations of wrongdoing by cops. outside the office, author...","criminal complaint: cop used his role to help cocaine traffickers .\nralph mata, an internal affairs lieutenant, allegedly helped group get guns .\nhe also arranged to pay two assassins in a murde..."
2,00027e965c8264c35cc1bc55556db388da82b07f,"a drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. craig eccleston-todd, 27, was driving home from a night at a pub when he r...","craig eccleston-todd, 27, had drunk at least three pints before driving car .\nwas using phone when he veered across road in yarmouth, isle of wight .\ncrashed head-on into 28-year-old rachel titl..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,"(cnn) -- with a breezy sweep of his pen president vladimir putin wrote a new chapter into crimea's turbulent history, committing the region to a future returned to russian domain. sixty years prio...","nina dos santos says europe must be ready to accept sanctions will hurt both sides .\ntargeting russia's business community would be one way of sapping their support for president putin, she says ..."
4,0003ad6ef0c37534f80b55b4235108024b407f0b,fleetwood are the only team still to have a 100% record in sky bet league one as a 2-0 win over scunthorpe sent graham alexander’s men top of the table. the cod army are playing in the third tier ...,"fleetwood top of league one after 2-0 win at scunthorpe .\npeterborough, bristol city, chesterfield and crawley all drop first points of the season .\nstand-in striker matt done scores a hat-trick..."


In [18]:
data_train['article']=data_train['article'].str.lower()

data_train['highlights']=data_train['highlights'].str.lower()

Converting to lower case

In [19]:
data_train.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"by . associated press . published: . 14:11 est, 25 october 2013 . | . updated: . 15:36 est, 25 october 2013 . the bishop of the fargo catholic diocese in north dakota has exposed potentially hundr...","bishop john folda, of north dakota, is taking time off after being diagnosed .\nhe contracted the infection through contaminated food in italy .\nchurch members in fargo, grand forks and jamestown..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(cnn) -- ralph mata was an internal affairs lieutenant for the miami-dade police department, working in the division that investigates allegations of wrongdoing by cops. outside the office, author...","criminal complaint: cop used his role to help cocaine traffickers .\nralph mata, an internal affairs lieutenant, allegedly helped group get guns .\nhe also arranged to pay two assassins in a murde..."
2,00027e965c8264c35cc1bc55556db388da82b07f,"a drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. craig eccleston-todd, 27, was driving home from a night at a pub when he r...","craig eccleston-todd, 27, had drunk at least three pints before driving car .\nwas using phone when he veered across road in yarmouth, isle of wight .\ncrashed head-on into 28-year-old rachel titl..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,"(cnn) -- with a breezy sweep of his pen president vladimir putin wrote a new chapter into crimea's turbulent history, committing the region to a future returned to russian domain. sixty years prio...","nina dos santos says europe must be ready to accept sanctions will hurt both sides .\ntargeting russia's business community would be one way of sapping their support for president putin, she says ..."
4,0003ad6ef0c37534f80b55b4235108024b407f0b,fleetwood are the only team still to have a 100% record in sky bet league one as a 2-0 win over scunthorpe sent graham alexander’s men top of the table. the cod army are playing in the third tier ...,"fleetwood top of league one after 2-0 win at scunthorpe .\npeterborough, bristol city, chesterfield and crawley all drop first points of the season .\nstand-in striker matt done scores a hat-trick..."


In [20]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [21]:
data_train['article'] = data_train['article'].apply(remove_html_tags)

data_train['highlights'] = data_train['highlights'].apply(remove_html_tags)

In [22]:
data_train.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"by . associated press . published: . 14:11 est, 25 october 2013 . | . updated: . 15:36 est, 25 october 2013 . the bishop of the fargo catholic diocese in north dakota has exposed potentially hundr...","bishop john folda, of north dakota, is taking time off after being diagnosed .\nhe contracted the infection through contaminated food in italy .\nchurch members in fargo, grand forks and jamestown..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(cnn) -- ralph mata was an internal affairs lieutenant for the miami-dade police department, working in the division that investigates allegations of wrongdoing by cops. outside the office, author...","criminal complaint: cop used his role to help cocaine traffickers .\nralph mata, an internal affairs lieutenant, allegedly helped group get guns .\nhe also arranged to pay two assassins in a murde..."
2,00027e965c8264c35cc1bc55556db388da82b07f,"a drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. craig eccleston-todd, 27, was driving home from a night at a pub when he r...","craig eccleston-todd, 27, had drunk at least three pints before driving car .\nwas using phone when he veered across road in yarmouth, isle of wight .\ncrashed head-on into 28-year-old rachel titl..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,"(cnn) -- with a breezy sweep of his pen president vladimir putin wrote a new chapter into crimea's turbulent history, committing the region to a future returned to russian domain. sixty years prio...","nina dos santos says europe must be ready to accept sanctions will hurt both sides .\ntargeting russia's business community would be one way of sapping their support for president putin, she says ..."
4,0003ad6ef0c37534f80b55b4235108024b407f0b,fleetwood are the only team still to have a 100% record in sky bet league one as a 2-0 win over scunthorpe sent graham alexander’s men top of the table. the cod army are playing in the third tier ...,"fleetwood top of league one after 2-0 win at scunthorpe .\npeterborough, bristol city, chesterfield and crawley all drop first points of the season .\nstand-in striker matt done scores a hat-trick..."


In [23]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [24]:
data_train['article'] = data_train['article'].apply(remove_punc)

data_train['highlights'] = data_train['highlights'].apply(remove_punc)